In [2]:
def complex_mult(a_r, a_i, b_r, b_i):
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_r * b_i + a_i * b_r
    return real_part, imag_part

def butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i):
    out1_r = in1_r + in2_r
    out1_i = in1_i + in2_i

    z1_r = in1_r - in2_r
    z1_i = in1_i - in2_i

    out2_r, out2_i = complex_mult(z1_r, z1_i, w_r, w_i)

    return out1_r, out1_i, out2_r, out2_i

def test_butterfly():
    in1_r = 10
    in1_i = 5
    in2_r = 3
    in2_i = 2
    w_r = 2
    w_i = 3

    out1_r, out1_i, out2_r, out2_i = butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i)

    print(f"out1_r: {out1_r}, out1_i: {out1_i}, out2_r: {out2_r}, out2_i: {out2_i}")

    in2_r = 8
    in2_i = 4
     
    out1_r, out1_i, out2_r, out2_i = butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i)

    print(f"out1_r: {out1_r}, out1_i: {out1_i}, out2_r: {out2_r}, out2_i: {out2_i}")
    
test_butterfly()

out1_r: 13, out1_i: 7, out2_r: 5, out2_i: 27
out1_r: 18, out1_i: 9, out2_r: 1, out2_i: 8


Butterfly works
<br />
iverilog output:

`
out1_r:    13, out1_i:     7, out2_r:     5, out2_i:    27
out1_r:    18, out1_i:     9, out2_r:     1, out2_i:     8
testbench.sv:58: $finish called at 20 (1s)
`

In [2]:
import numpy as np

def complex_mult(a_r, a_i, b_r, b_i):
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_r * b_i + a_i * b_r
    return real_part, imag_part

def butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i):
    out1_r = in1_r + in2_r
    out1_i = in1_i + in2_i

    z1_r = in1_r - in2_r
    z1_i = in1_i - in2_i

    out2_r, out2_i = complex_mult(z1_r, z1_i, w_r, w_i)

    return out1_r, out1_i, out2_r, out2_i

def four_point_fft(x_r, x_i):
    stage1_0_r, stage1_0_i, stage1_1_r, stage1_1_i = butterfly(x_r[0], x_i[0], x_r[2], x_i[2], 1, 0)
    stage1_2_r, stage1_2_i, stage1_3_r, stage1_3_i = butterfly(x_r[1], x_i[1], x_r[3], x_i[3], 1, 0)

    y0_r, y0_i, y2_r, y2_i = butterfly(stage1_0_r, stage1_0_i, stage1_2_r, stage1_2_i, 1, 0)
    y1_r, y1_i, y3_r, y3_i = butterfly(stage1_1_r, stage1_1_i, stage1_3_r, stage1_3_i, 0, -1)

    return [y0_r, y1_r, y2_r, y3_r], [y0_i, y1_i, y2_i, y3_i]

def test_four_point_fft():
    x_r = [10, 8, 6, 4]
    x_i = [0, 0, 0, 0]

    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    y_r, y_i = four_point_fft(x_r, x_i)

    print("Test Case 1:")
    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

    x_r = [10, 8, 6, 4]
    x_i = [5, 4, 3, 2]

    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    y_r, y_i = four_point_fft(x_r, x_i)

    print("\nTest Case 2:")
    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

test_four_point_fft()

Test Case 1:
Expected y0_r: 28.00, y0_i: 0.00
Computed y0_r: 28.00, y0_i: 0.00
Expected y1_r: 4.00, y1_i: -4.00
Computed y1_r: 8.00, y1_i: 0.00
Expected y2_r: 4.00, y2_i: 0.00
Computed y2_r: 4.00, y2_i: 0.00
Expected y3_r: 4.00, y3_i: 4.00
Computed y3_r: 0.00, y3_i: 0.00

Test Case 2:
Expected y0_r: 28.00, y0_i: 14.00
Computed y0_r: 28.00, y0_i: 14.00
Expected y1_r: 6.00, y1_i: -2.00
Computed y1_r: 8.00, y1_i: 4.00
Expected y2_r: 4.00, y2_i: 2.00
Computed y2_r: 4.00, y2_i: 2.00
Expected y3_r: 2.00, y3_i: 6.00
Computed y3_r: 0.00, y3_i: 0.00


Test Case 1:
Expected y0_r: 28.00, y0_i: 0.00
Computed y0_r: 28.00, y0_i: 0.00
Expected y1_r: 4.00, y1_i: -4.00
Computed y1_r: 8.00, y1_i: 0.00
Expected y2_r: 4.00, y2_i: 0.00
Computed y2_r: 4.00, y2_i: 0.00
Expected y3_r: 4.00, y3_i: 4.00
Computed y3_r: 0.00, y3_i: 0.00

Test Case 2:
Expected y0_r: 28.00, y0_i: 14.00
Computed y0_r: 28.00, y0_i: 14.00
Expected y1_r: 6.00, y1_i: -2.00
Computed y1_r: 8.00, y1_i: 4.00
Expected y2_r: 4.00, y2_i: 2.00
Computed y2_r: 4.00, y2_i: 2.00
Expected y3_r: 2.00, y3_i: 6.00
Computed y3_r: 0.00, y3_i: 0.00


In [7]:
import numpy as np

def complex_mult(a_r, a_i, b_r, b_i):
    real_part = a_r * b_r - a_i * b_i
    imag_part = a_r * b_i + a_i * b_r
    return real_part, imag_part

def butterfly(in1_r, in1_i, in2_r, in2_i, w_r, w_i):
    out1_r = in1_r + in2_r
    out1_i = in1_i + in2_i

    z1_r = in1_r - in2_r
    z1_i = in1_i - in2_i

    out2_r, out2_i = complex_mult(z1_r, z1_i, w_r, w_i)

    return out1_r, out1_i, out2_r, out2_i

def four_point_fft(x_r, x_i):
    stage1_0_r, stage1_0_i, stage1_2_r, stage1_2_i = butterfly(x_r[0], x_i[0], x_r[2], x_i[2], 1, 0)
    stage1_1_r, stage1_1_i, stage1_3_r, stage1_3_i = butterfly(x_r[1], x_i[1], x_r[3], x_i[3], 1, 0)

    y0_r, y0_i, y2_r, y2_i = butterfly(stage1_0_r, stage1_0_i, stage1_1_r, stage1_1_i, 1, 0)
    y1_r, y1_i, y3_r, y3_i = butterfly(stage1_2_r, stage1_2_i, stage1_3_r, stage1_3_i, 0, -1)

    return [y0_r, y1_r, y2_r, y3_r], [y0_i, y1_i, y2_i, y3_i]

def test_four_point_fft():
    x_r = [10, 8, 6, 4]
    x_i = [0, 0, 0, 0]

    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    y_r, y_i = four_point_fft(x_r, x_i)

    print("Test Case 1:")
    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

    x_r = [10, 8, 6, 4]
    x_i = [5, 4, 3, 2]

    x_complex = np.array(x_r) + 1j * np.array(x_i)
    fft_result = np.fft.fft(x_complex)

    y_r, y_i = four_point_fft(x_r, x_i)

    print("\nTest Case 2:")
    for i in range(4):
        print(f"Expected y{i}_r: {fft_result[i].real:.2f}, y{i}_i: {fft_result[i].imag:.2f}")
        print(f"Computed y{i}_r: {y_r[i]:.2f}, y{i}_i: {y_i[i]:.2f}")

# Run the test cases
test_four_point_fft()

Test Case 1:
Expected y0_r: 28.00, y0_i: 0.00
Computed y0_r: 28.00, y0_i: 0.00
Expected y1_r: 4.00, y1_i: -4.00
Computed y1_r: 8.00, y1_i: 0.00
Expected y2_r: 4.00, y2_i: 0.00
Computed y2_r: 4.00, y2_i: 0.00
Expected y3_r: 4.00, y3_i: 4.00
Computed y3_r: 0.00, y3_i: 0.00

Test Case 2:
Expected y0_r: 28.00, y0_i: 14.00
Computed y0_r: 28.00, y0_i: 14.00
Expected y1_r: 6.00, y1_i: -2.00
Computed y1_r: 8.00, y1_i: 4.00
Expected y2_r: 4.00, y2_i: 2.00
Computed y2_r: 4.00, y2_i: 2.00
Expected y3_r: 2.00, y3_i: 6.00
Computed y3_r: 0.00, y3_i: 0.00


In [8]:
import numpy as np

x = np.array([1000, 2000, 3000, 4000], dtype=np.complex64)

y = np.fft.fft(x)

def format_complex(c):
    return f"{int(c.real):d} + j{int(c.imag):d}"

print("Inputs:")
for i, val in enumerate(x):
    print(f"x{i} = {format_complex(val)}")

print("\nOutputs:")
for i, val in enumerate(y):
    print(f"y{i} = {format_complex(val)}")

print("\nMagnitude and Phase:")
for i, val in enumerate(y):
    mag = np.abs(val)
    phase = np.angle(val, deg=True)
    print(f"y{i}: magnitude = {mag:.2f}, phase = {phase:.2f} degrees")

Inputs:
x0 = 1000 + j0
x1 = 2000 + j0
x2 = 3000 + j0
x3 = 4000 + j0

Outputs:
y0 = 10000 + j0
y1 = -2000 + j2000
y2 = -2000 + j0
y3 = -2000 + j-2000

Magnitude and Phase:
y0: magnitude = 10000.00, phase = 0.00 degrees
y1: magnitude = 2828.43, phase = 135.00 degrees
y2: magnitude = 2000.00, phase = 180.00 degrees
y3: magnitude = 2828.43, phase = -135.00 degrees
